In [1]:
import pandas as pd
import csv
import matplotlib.pyplot as plt
from itertools import combinations

In [2]:
ratings_1 = "combined_data_1.txt"
ratings_2 = "combined_data_2.txt"
ratings_3 = "combined_data_3.txt"
ratings_4 = "combined_data_4.txt"

# Not loading all the data at once to reduce iteration time
# ratings = [ratings_1, ratings_2, ratings_3, ratings_4]
ratings = [ratings_1]
movie_titles = "movie_titles.csv"
dataset_path = "./data"


### Load and preprocess movies and ratings data


The ratings txt file is structured as follows:
moveid id, colon

> 1:

then the ratings for that movie in the format:
customer_id rating date

> 1488844 3 2005-09-06


as such, we need to process a little more manually than pd.read_csv

In [3]:
def load_ratings_data(filename):
    data = []
    with open(filename, 'r') as file:
        current_movie_id = None
        for line in file:
            line = line.strip()
            if line.endswith(':'):
                current_movie_id = int(line.replace(':', ''))
            else:
                customer_id, rating, date = line.split(',')
                data.append([current_movie_id, int(customer_id), float(rating), date])
    return pd.DataFrame(data, columns=['Movie_Id', 'Cust_Id', 'Rating', 'Date'])


In [4]:
df_ratings = pd.concat([load_ratings_data(f"{dataset_path}/{rating}") for rating in ratings])


FileNotFoundError: [Errno 2] No such file or directory: './data/combined_data_1.txt'

In [ ]:
df_ratings.head(3)

In [ ]:
print(df_ratings.shape)

In [ ]:
df_ratings.describe()

In [ ]:
cust_count = df_ratings['Cust_Id'].nunique()
movie_count = df_ratings['Movie_Id'].nunique()
rating_count = df_ratings['Cust_Id'].count()


print(f"Total number of unique customers: {cust_count}")

p = df_ratings.groupby('Rating')['Rating'].agg(['count'])

ax = p.plot(kind = 'barh', legend = False, figsize = (15,10))
plt.title('Total pool: {:,} Movies, {:,} customers, {:,} ratings given'.format(movie_count, cust_count, rating_count), fontsize=20)
plt.axis('off')

for i in range(1,6):
    ax.text(p.iloc[i-1][0]/4, i-1, 'Rating {}: {:.0f}%'.format(i, p.iloc[i-1][0]*100 / p.sum()[0]), color = 'white', weight = 'bold')

In [ ]:
titles = []

with open(f"{dataset_path}/{movie_titles}", encoding="ISO-8859-1") as file:
    reader = csv.reader(file)
    for row in reader:
        movie_id = int(row[0])
        # Need to handle a few cases where the year is missing. 
        if row[1] == 'NULL':
            year = -1
        else:
            year = int(row[1])
        # Need to handle the case where a movie title has a comman in the name
        name = ','.join(row[2:]) 
        titles.append([movie_id, year, name])

df_titles = pd.DataFrame(titles, columns=['Movie_Id', 'Movie_Year', 'Name'])
df_titles.set_index('Movie_Id', inplace=True)


In [ ]:
df_titles.head(3)

In [ ]:
df = df_ratings.join(df_titles, on='Movie_Id', how='inner')

In [ ]:
df.head()

In [ ]:
# No missing values

# na_check = df.isna().sum()
# print(na_check)

### Enrich data with average rating and review count for each movie

In [ ]:
average_ratings = df.groupby('Movie_Id')['Rating'].mean().reset_index()
average_ratings.columns = ['Movie_Id', 'Average_Rating']

review_counts = df.groupby('Movie_Id')['Rating'].count().reset_index()
review_counts.columns = ['Movie_Id', 'Review_Count']

In [ ]:
df = df.merge(average_ratings, on='Movie_Id', how='inner')
df = df.merge(review_counts, on='Movie_Id', how='inner')

### Find set of N movies where the customers have rate all N movies 

In [ ]:
KEEP_TOP_N = 20
top_movies = review_counts.sort_values(by='Review_Count', ascending=False)['Movie_Id']
top_n_movies = top_movies.head(KEEP_TOP_N)

In [ ]:
df_top_n = df[df['Movie_Id'].isin(top_n_movies)]
print(df_top_n.shape)

In [ ]:
pivot_table = df_top_n.pivot_table(index='Cust_Id', columns='Movie_Id', values='Rating', aggfunc='count', fill_value=0)


In [ ]:
customers_all_n_movies = pivot_table[pivot_table.sum(axis=1) == KEEP_TOP_N].index


In [ ]:
df_final = df_top_n[df_top_n['Cust_Id'].isin(customers_all_n_movies)]


In [ ]:
df_final.head()

In [ ]:
print(df_final.shape)

In [ ]:
print(f"Number of customers who reviewed all {KEEP_TOP_N} movies: {len(customers_all_n_movies)}")
print(df_titles.loc[top_n_movies])


In [ ]:
df_final.shape

### Save Dataframes

In [ ]:
df_final.to_csv(f"{dataset_path}/df_top_20_movies_customers_reviewed_all.csv", index=False)

In [ ]:
df.to_csv(f"{dataset_path}/df_all_enriched.csv", index=False)